Using VQE, To predict the lowest eigen value for H3

In [1]:
import pennylane as qml 
from pennylane import numpy as np
import pennylane.qchem as qchem

First, we need to define the molecule we want and it's geometrical structure

In [2]:
symbols = ["H", "H", "H"]
coordinates = np.array([
    [0.0102, 0.0442, 0.0],
    [0.9867, 1.6303, 0.0],
    [1.8720, -0.0085, 0.0]
])


Construct the hamoltonian operator 

In [3]:
hamoltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge = 1)

Intializing the reference state using Hartee Fock approxamiation 

In [4]:
hf = qchem.hf_state(electrons=2, orbitals=6)

In [5]:
hf

array([1, 1, 0, 0, 0, 0])

In [6]:
num_wires = qubits
dev = qml.device("default.qubit", wires= num_wires)

In [7]:
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(hamoltonian)

Predicting the lowest bound eigen value for our quantum state using Hartee Fock Approximation

In [8]:
exp_energy(hf)

c:\Users\20102\miniconda3\envs\quantum\Lib\site-packages\pennylane\ops\op_math\composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


tensor(-1.24655016, requires_grad=True)

Creating our variational circuit form

In [9]:
def ansatz(param):
    qml.BasisState(hf, wires=range(num_wires))
    qml.DoubleExcitation(param[0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(param[1], wires=[0, 1, 4, 5])
    

Creating the cost function based on the expectation value from getting the expectancy of the hamoltonian

In [10]:
@qml.qnode(dev)
def cost_function(param):
    ansatz(param)
    return qml.expval(hamoltonian)

In [11]:
cost_function([0.1 , 0.1])

tensor(-1.26796721, requires_grad=True)

Creating the optimization process using Gradient Descent to minmize our objective function which is the expectation value of Hamoltonian

In [12]:
ops = qml.GradientDescentOptimizer(stepsize=0.4)

theta = np.array([0.0, 0.0], requires_grad=True)

energy = [cost_function(hf)]
angle = [theta]
max_iterations = 20

for n in range(max_iterations):
    theta, prev_energy = ops.step_and_cost(cost_function, theta)
    energy.append(cost_function(theta))
    
    if n%2 == 0:
        print(f" Step = {n}, Energy = {energy[-1]:.8f} Ha")


 Step = 0, Energy = -1.26070025 Ha
 Step = 2, Energy = -1.27115671 Ha
 Step = 4, Energy = -1.27365804 Ha
 Step = 6, Energy = -1.27425241 Ha
 Step = 8, Energy = -1.27439362 Ha
 Step = 10, Energy = -1.27442718 Ha
 Step = 12, Energy = -1.27443517 Ha
 Step = 14, Energy = -1.27443707 Ha
 Step = 16, Energy = -1.27443752 Ha
 Step = 18, Energy = -1.27443763 Ha


In [13]:
print(f"Final ground energy: {energy[-1]:.8f} Ha")
print(f"Final angle parameters: {theta[0]:.8f} {theta[1]:.8f}")

Final ground energy: -1.27443764 Ha
Final angle parameters: 0.19203468 0.19290335
